In [1]:
# coding: utf-8
# ============================================================================
#   Copyright (C) 2017 All rights reserved.
#
#   filename : Assignment_template.py
#   author   : chendian / okcd00@qq.com
#   date     : 2018-11-15
#   desc     : Tensorflow Tuple Extraction Tutorial
# ============================================================================

from __future__ import print_function
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""
# os.environ["CUDA_VISIBLE_DEVICES"] = "2" (if you want to use GPU2)

import sys
import math
import json
import pickle
import numpy as np

In [2]:
# How to load training data in Python:
import json
sentence_list_fname = './assignment_test_data_word_segment.json'
sentence_list = json.load(open(sentence_list_fname , 'r',encoding='utf-8'))

# How to load vocabulary in Python:

In [3]:
sentence_array = []
unicode_word = 0
sentence = sentence_list[4]
for index in range(20):
    print(sentence_list[index])

sentence_times = sentence['times']
sentence_values = sentence['values']
sentence_attributes = sentence['attributes']
result_rule = []
result_rule_index = []
count_index = 0
for word in sentence['indexes']:
    if word == 0:
        result_rule.append('T')
        result_rule_index.append(count_index)
    if word == 2:
        result_rule.append('V')
        result_rule_index.append(count_index)
    if word == 1:
        result_rule.append('A')
        result_rule_index.append(count_index)
    count_index += 1
print(result_rule)  
print(result_rule_index)  

{'sentenceId': '76d78d5af70bb6608fd5ef731922e6036eda6d52-20248dad98e94d3180de3d525e74fc7ba0cea465', 'sentence': '截至2015年12月31日，碧桂园控股累计对外担保金额约为6,241,806万元，剔除为物业买方向银行申请按揭贷款而提供的短期过渡性担保，碧桂园控股累计对外担保金额约为178,190万元，仅占期末归属于母公司所有者权益的2.73%。', 'times': [1], 'values': [10, 35, 45], 'words': ['截至', '2015年12月31日', '，', '碧桂园', '控股', '累计', '对外担保', '金额', '约', '为', '6,241,806万元', '，', '剔除', '为', '物业', '买方', '向', '银行', '申请', '按揭', '贷款', '而', '提供', '的', '短期', '过渡性', '担保', '，', '碧桂园', '控股', '累计', '对外担保', '金额', '约', '为', '178,190万元', '，', '仅', '占', '期末', '归属', '于', '母公司', '所有者权益', '的', '2.73%', '。'], 'attributes': [6, 31, 43], 'indexes': [15, 0, 7, 677, 256, 315, 1, 51, 357, 8, 2, 7, 1108, 8, 223, 2316, 462, 149, 2557, 1236, 209, 186, 248, 12, 105, 2691, 203, 7, 677, 256, 315, 1, 51, 357, 8, 2, 7, 764, 14, 41, 302, 112, 94, 1, 12, 2, 10]}
{'sentenceId': 'e71826135a20306ebc52acba1c5c793716c8d81a-8664c9cd649bfb47ce153b98d0ff6b1845e71629', 'sentence': '2012年度、2013年度、2014年度和2015年1-9月，公司营业外支出分别为349.07万元、164.73万元、

In [4]:
special_array = [88,14,19,97,21,360,22,40,1796,309,102]
cut_char_array = [7,10,26]
change_word = [22,32,12,261,546,49]
change_s_word = [141,644]

def check_result(sen_index):
    sentence = sentence_list[sen_index]
    results = []
    for time in sentence['times']:
        for attribute in sentence['attributes']:
            for value in sentence['values']:
                results.append([time, attribute, value])
    indexes = sentence['indexes']
    for index in range(len(indexes) - 3):
        if (indexes[index] == 2) and (indexes[index + 1] == 7) and (indexes[index + 2] == 2) :
            indexes[index + 1] = 6
        if (indexes[index] == 7) and (indexes[index + 1] == 11) and (indexes[index + 2] == 8) :
            indexes[index] = 6
            
    final_result = []
    not_in_ssen = 0
    for result in results:
        #过滤V
        if indexes[result[2] - 1] in change_word:
            continue
        if indexes[result[2] + 1] in change_s_word:
            continue
        
        
        #顺序TAV
        #if result[0] >=  result[1]:
         #   continue
        if result[1] >=  result[2]:
            continue
             
        #过滤AV不在同一个短语的
        counter = result[1]
        while counter <= result[2]:
            if (indexes[counter] in cut_char_array):
                #if (indexes[counter] == 7 and result[2] <= counter + 3):
                 #   continue
                not_in_ssen = 1
                break
            counter += 1
            
        if not_in_ssen == 1:
            not_in_ssen = 0
            continue
        
        #过滤短语内有特殊内容的
        counterb = result[1]
        while 1:
            counterb -= 1
            if indexes[counterb] in cut_char_array:       
                break
            if counterb == 0 or counterb == -1:
                break
        countere = result[2]
        while 1:
            countere += 1
            if (indexes[countere] in cut_char_array):
               # if (indexes[countere] == 7 and result[2] <= countere + 3):
                #    continue
                break
                
        special_char = 0
        counter = counterb
        while counter <= countere:
            if indexes[counter] in special_array:  
                special_char = 1
                break
            counter += 1        
        if special_char == 1:
            continue   
            
        #处理T和AV的对应关系
        num_v = 0
        res_v_index = 0    
        counter = counterb
        while counter <= countere:
            if indexes[counter] == 2:       
                num_v += 1
            if counter == result[2]: 
                res_v_index = num_v
            counter += 1
        search_t_index = num_v - res_v_index
        counter = result[1]
        result_t = 0
        while counter >= 0:
            if indexes[counter] == 0:  
                if search_t_index <= 0:
                    result_t = counter
                    break
                else :
                    search_t_index -= 1
            counter -= 1  
        if result[0] != result_t:
            continue
        
        final_result.append(result)
    counter = 0
    while counter < (len(final_result) - 1):
        if final_result[counter][2] == final_result[counter + 1][2]:  
            del final_result[counter]
        else:
            counter += 1
    return final_result  

测试集：
以下为测试集部分，输出output_test.json，其中在原有assignment_test_data_word_segment.json基础上增加了results项，即为预测的结果。

In [5]:
test_sentence_list_fname = './assignment_test_data_word_segment.json'
test_sentence_list = json.load(open(test_sentence_list_fname , 'r',encoding='utf-8'))
test_final_result = []
for i in range(len(test_sentence_list)): 
    test_final_result.append(check_result(i))
print(test_final_result)

[[[1, 6, 10], [1, 31, 35]], [[0, 9, 12], [2, 9, 14], [4, 9, 16], [6, 9, 18]], [[0, 9, 12], [0, 21, 24], [2, 9, 14], [2, 21, 26], [4, 9, 16], [4, 21, 28], [6, 9, 18], [6, 21, 30]], [[0, 9, 12], [2, 9, 14], [4, 9, 16], [6, 9, 18]], [], [[1, 5, 7]], [[1, 8, 11]], [[0, 12, 15], [2, 12, 17], [4, 12, 19], [6, 12, 21]], [[0, 20, 23], [2, 20, 25], [4, 20, 27]], [[1, 4, 6]], [[1, 5, 6], [1, 8, 9], [1, 11, 12], [14, 16, 17], [14, 19, 20], [14, 22, 23]], [[1, 5, 6], [1, 8, 9], [1, 11, 12], [14, 16, 17], [14, 19, 20], [14, 22, 23]], [[1, 5, 6], [1, 8, 9], [1, 11, 12], [14, 16, 17], [14, 19, 20], [14, 22, 23]], [[1, 5, 6], [1, 8, 9], [1, 11, 12], [14, 16, 17], [14, 19, 20], [14, 22, 23]], [[1, 5, 6], [1, 8, 9], [1, 11, 12], [14, 16, 17], [14, 19, 20], [14, 22, 23]], [[1, 5, 6], [1, 8, 9], [1, 11, 12], [14, 16, 17], [14, 19, 20], [14, 22, 23]], [[1, 5, 6], [1, 8, 9], [1, 11, 12], [14, 16, 17], [14, 19, 20], [14, 22, 23]], [[1, 5, 6], [1, 8, 9], [1, 11, 12], [14, 16, 17], [14, 19, 20], [14, 22, 23]],

In [6]:
for i in range(len(test_sentence_list)): 
    test_sentence_list[i]['results'] = test_final_result[i]
file_out_test = open("output_test.json", "w")
file_out_test.write(json.dumps(test_sentence_list))
for i in range(20): 
    print(test_sentence_list[i])
file_out_test.close()

{'sentenceId': '76d78d5af70bb6608fd5ef731922e6036eda6d52-20248dad98e94d3180de3d525e74fc7ba0cea465', 'sentence': '截至2015年12月31日，碧桂园控股累计对外担保金额约为6,241,806万元，剔除为物业买方向银行申请按揭贷款而提供的短期过渡性担保，碧桂园控股累计对外担保金额约为178,190万元，仅占期末归属于母公司所有者权益的2.73%。', 'times': [1], 'values': [10, 35, 45], 'words': ['截至', '2015年12月31日', '，', '碧桂园', '控股', '累计', '对外担保', '金额', '约', '为', '6,241,806万元', '，', '剔除', '为', '物业', '买方', '向', '银行', '申请', '按揭', '贷款', '而', '提供', '的', '短期', '过渡性', '担保', '，', '碧桂园', '控股', '累计', '对外担保', '金额', '约', '为', '178,190万元', '，', '仅', '占', '期末', '归属', '于', '母公司', '所有者权益', '的', '2.73%', '。'], 'attributes': [6, 31, 43], 'indexes': [15, 0, 7, 677, 256, 315, 1, 51, 357, 8, 2, 7, 1108, 8, 223, 2316, 462, 149, 2557, 1236, 209, 186, 248, 12, 105, 2691, 203, 7, 677, 256, 315, 1, 51, 357, 8, 2, 7, 764, 14, 41, 302, 112, 94, 1, 12, 2, 10], 'results': [[1, 6, 10], [1, 31, 35]]}
{'sentenceId': 'e71826135a20306ebc52acba1c5c793716c8d81a-8664c9cd649bfb47ce153b98d0ff6b1845e71629', 'sentence': '2012年度、2013年度、2014年度和